Functions in this notebook:


*   Extracting Longitude and Latitude from google maps API generated JSON data type
*   Mapping each participant as a graph of geographic school history
*   Mapping each participant as a graph of geographic school history, but for a subset of ID's
*   Mapping each participant as a graph of geographic school history, but color coded to compare two groups
*   Finding ratio of geocodable ID's to total ID's of a subset
*   Finding geodesic and binary path length
*   Making subset of path lengths based on redcap generated report
*   Pulling census data for neighborhood level measures of median income, and creating maps that vizualize income changes










# Import Statements

In [ ]:

# for secure API storage
!pip install "apikey>=0.2.1"
!pip install git+ssh://git@github.com/ulf1/apikey.git
import apikey

# for geocoding
!pip install -U googlemaps
import googlemaps
from datetime import datetime
import geopy
from geopy import distance

# for data
import numpy as np
import pandas as pd

## for plotting
!pip install seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## for machine learning
from sklearn import preprocessing, cluster
import scipy

# for map making
!pip install folium
import folium
import branca
import matplotlib.colors

# for census data
!pip install us
!pip install census
!pip install censusgeocode
import censusgeocode as cg
from census import Census
from us import states

# other useful tools
from pandas.core.arrays.numeric import T
import random
from math import sqrt, cos, radians
from math import atan
import copy 
import csv

# improving visibility of dataframes
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)




 To load past geocoded and census mapped data, uncomment the following and skip section 1 

In [ ]:
# pathe = path to directory where MBES data folder is stored 
# df = pd.read_csv(pathe + 'MBES/ElemGeoCode.csv')
# dftract = pd.read_csv(pathe + 'MBES/ElemCensusTract.csv')

# 1. Geocoding and preprocessing

In [ ]:
# pathe = path to directory where MBES data folder is stored 
# Load keys for google maps API and census API into apikey file
#(Code not included, see https://pypi.org/project/apikey/)
googkey = apikey.load('google', filename= pathe + 'MBES/secretkey')
gmaps = googlemaps.Client(key=googkey)
dtf = pd.read_csv( pathe + 'MBES/MBES_List_Of_All_Schools.csv')
classify = pd.read_csv( pathe + 'MBES/MBES_List_Of_All_Schools.csv')
classify.drop(columns = ['AGE CHECK','GRADE CHECK','study_check', 'NOTES ON WHY CHANGES WERE MADE -CS','Study ID.1'], inplace = True)
classify.replace({ r'\A\s+|\s+\Z': '', '\n' : ' '}, regex=True, inplace=True)
# Drop rows with blank addresses
dtf.dropna(axis = 0, subset = 'Address', inplace = True)

# Drop extra columns 
dtf.drop(columns = ['AGE CHECK','GRADE CHECK','study_check', 'NOTES ON WHY CHANGES WERE MADE -CS'], inplace = True)
dtf.drop(columns = 'Study ID.1' , inplace = True)

# Replace line breaks with spaces
dtf.replace({ r'\A\s+|\s+\Z': '', '\n' : ' '}, regex=True, inplace=True)

print('The total number of schools are: ' + str(len(dtf)))
print('The total number of participants with atleast 1 address: ' + str(len(dtf['Study ID'].unique())))

In [ ]:
# The following method classifies each address based on quality,

# Full Address with st # - 5 (Best)
# Street + zipcode (missing st #) - 4
# Zip Code - 3
# State & City - 2
# State only - 1 (Worst)
classify.insert(7, 'Geolocation Confidence Level (1 - 5)', None)

In [ ]:
classify = classify.replace(np.nan, None)
classify.head(500)

In [ ]:
def classifyAddress(df):
    for i in range(0,len(df.index)):
            if (df.iloc[i,6] is None) & (df.iloc[i,5] is None) & (df.iloc[i,3] is None) & (df.iloc[i,4] is not None):
                df.iloc[i,7] = 1
     
            if (df.iloc[i,6] is None) & (df.iloc[i,5] is None) & (df.iloc[i,3] is not None) & (df.iloc[i,4] is not None):
                df.iloc[i,7] = 2
            if (df.iloc[i,6] is not None):
                df.iloc[i,7] = 3
            if (df.iloc[i,6] is not None) & (df.iloc[i,5] is not None):
                df.iloc[i,7] = 5
        
    return df

In [ ]:

classify = classifyAddress(classify)

In [ ]:
classify.to_csv(pathe+'MBES/ElemConfidenceAnalysis.csv', index = False)

In [ ]:
# Geocoding data using google maps API
dtf['location'] = dtf['Address'].apply(gmaps.geocode)

In [ ]:
print('After Geocoding...')
print('The total number of schools are: ' + str(len(dtf)))
print('The total number of participants with atleast 1 address: ' + str(len(dtf['Study ID'].unique())))

In [ ]:
pd.set_option('display.max_rows', 500)
# Function to extract longitude and latidue from Google Maps API generated JSON format
def latlong(df):
   dtf['lat'] = None
   dtf['long'] = None
   numRows = len(df.index)
   for f in range(0,numRows):
      if df.iloc[f,19]:

        df.iloc[f,20]= df.iloc[f,19][0].get('geometry').get('location').get('lat')
        df.iloc[f,21]= df.iloc[f,19][0].get('geometry').get('location').get('lng')

   return df

dtf = latlong(dtf)


In [ ]:
# drop rows with blank latitude values
df = dtf.dropna(axis = 0, subset = 'lat')

len(df[df['lat'].isna() == True])
df.to_csv(pathe + 'MBES/ElemGeoCode.csv', index = False)

In [ ]:
print('After Geocoding and Dropping Blanks...')
print('The total number of schools are: ' + str(len(dtf)))
print('The total number of participants with atleast 1 address: ' + str(len(dtf['Study ID'].unique())))

Now, we will geocode lat/long into census tract for use in the fourth section. Skip to section 2 if not needed

In [ ]:
key = apikey.load("census", filename=pathe + 'MBES/secretkey')
c = Census(key)
df['censusloc'] = df.apply(lambda x: cg.coordinates(x= x.long, y=x.lat), axis=1)

In [ ]:
# Function to extract longitude and latidue from Google Maps API generated JSON format
def tractinfo(df):
   df['Tract'] = None
   df['State_Fips'] = None
   df['County_Fips'] = None
   numRows = len(df.index)
   for f in range(0,numRows):
        if df.iloc[f,22]:
            df.iloc[f,23] = df.iloc[f,22].get('2020 Census Blocks')[0].get('TRACT')
            df.iloc[f,24] = df.iloc[f,22].get('2020 Census Blocks')[0].get('STATE')
            df.iloc[f,25] = df.iloc[f,22].get('2020 Census Blocks')[0].get('COUNTY')
   return df

df = tractinfo(df)

In [ ]:
print('The number of addresses without Census Tracts are: ' + str(len(df[df['Tract'].isna() == True])) )

In [ ]:
# Since only US addresses can be used, lets drop the addresses where no tract could be found 
dftract = df.dropna(axis = 0, subset = 'Tract')   

In [ ]:
print('The number of addresses without Census Tracts are: ' + str(len(df[df['Tract'].isna() == True])) )

In [ ]:
# Pulling the median income from Census api based on tract 
dftract['Median Income'] = dftract.apply(lambda x: c.acs5.state_county_tract(fields = 'B19126_001E', state_fips = x.State_Fips, county_fips = x.County_Fips, tract = x.Tract), axis=1)
# Getting numerical value of median income 
dftract['Median Income Number'] = dftract['Median Income'].apply(lambda x: x[0].get('B19126_001E'))
# dropping tracts with median income of 0 or less
dftract = dftract[dftract['Median Income Number'] > 0]


In [ ]:
def incomechange(df):
    #Creates column calculating for each address the change in median income
    #following the previous address
    df['Income Change'] = None
    unique = df['Study ID'].unique()
    numRows = len(df.index)
    for f in range(0,numRows):
        if df.iloc[f,1] == 1: 
            df.iloc[f,28] = 0 
        else: 
            df.iloc[f,28] = df.iloc[f,27] - df.iloc[f-1,27]
    return df
dftract = incomechange(dftract)

In [ ]:
# Saving data to reduce processing time in the future 
dftract.to_csv(pathe + 'MBES/ElemCensusTract.csv', index = False)

# 2. Functions to make maps and examples

In [ ]:
# Map1 plots each address on map
map1 = folium.Map(
    location=[37.0902, -95.7129],
     height=1000,
    zoom_start=5,
)
df.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]], tooltip = (str(int(row['Study ID'])) + ' ' + str(row['School Name']))).add_to(map1), axis=1)

# Now, we will connect each address into a graph 

# Function to add a line corresponding to each Study ID
# Color pallete that will be used to randomly assign colors to each participants path 
colorPal = ['#ffe119', '#4363d8', '#f58231','#9A6324','#f032e6','#e6194B' ,'#42d4f4','#dcbeff', '#800000', '#000075', '#a9a9a9','#3cb44b', '#000000']
def graphmap(df):
  map2 = folium.Map(
    location=[37.0902, -95.7129],
     height=1000,
    zoom_start=5,
)
  unique = df['Study ID'].unique()
  for i in unique:
    s = df[df['Study ID'] == i]
    numRows = len(s)

    graphpoints = []
    for f in range(0,numRows):
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]

      tup = [lat,long]

      graphpoints.append(tup)
    folium.PolyLine(locations = graphpoints, tooltip = s.iloc[f,0], color = random.choice(colorPal)).add_to(map2)
  return map2
# For graphing a map using a specific set of study ID's
def searchgraphmap(df,subset):
  map2 = folium.Map(
    location=[37.0902, -95.7129],
     height=1000,
    zoom_start=5,
)
  colorPal = ['#ffe119', '#4363d8', '#f58231','#9A6324','#f032e6','#e6194B' ,'#42d4f4','#dcbeff', '#800000', '#000075', '#a9a9a9','#3cb44b', '#000000']
 
  for i in subset:

    s = df[df['Study ID'] == i]

    numRows = len(s)
    if numRows == 0:
      continue
    graphpoints = []
    for f in range(0,numRows):
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]

      tup = [lat,long]

      graphpoints.append(tup)
    folium.PolyLine(locations = graphpoints, color = random.choice(colorPal), tooltip = s.iloc[f,0]).add_to(map2)
  return map2

graphm = graphmap(df)

# Compute ratio of geocodable ID's to total ID's in subset
def geocodedratio(df, subset):
  count = 0
  for i in subset:
    if len(df[df['Study ID'] == i]) != 0:
      count = count + 1
  ratio = count/len(subset)
  return ratio


Map of all participants with atleast one geocodable address (N = 120)

In [ ]:
graphm

In [ ]:

with open(pathe + 'MBES/Low_Income_Study_ID.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    low = list(csv_reader)
with open(pathe + 'MBES/Mid_Income_Study_ID.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    mid = list(csv_reader)
m = []
for i in mid:
  m.append(int(i[0]))
l = []
for i in low:
  l.append(int(i[0]))


Search Graph Map function allows generation of subset of map with one line of code

In [ ]:

midmap = searchgraphmap(df,m)
lowmap = searchgraphmap(df,l)


Participants between 80% and 200% of median county income (Middle income)

In [ ]:
print("The geocoded ratio of middle income participants is " + str(geocodedratio(df, m)))

In [ ]:

midmap

Participants below 80% of county median

In [ ]:
print("The geocoded ratio of low income participants is " + str(geocodedratio(df, l)))

The geocoded ratio is useful when commparing two sets, to see if group differences might be effected by differences in how many participants in each group had addresses that could be located on a map

In [ ]:
lowmap

A function that generates a map that compares two lists of Study ID's

In [ ]:
def compareGroups(df,subset1, subset2, title1, title2):
  map2 = folium.Map(
    location=[37.0902, -95.7129],
     height=1000,
    zoom_start=5,
)
  legend_html = '''
{% macro html(this, kwargs) %}
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 250px;
    height: 80px;
    z-index:9999;
    font-size:14px;
    ">
    <p><a style="color:#4363d8;font-size:150%;margin-left:20px;">&diams;</a>&emsp;''' + title1 + '''</p>
    <p><a style="color:#f032e6;font-size:150%;margin-left:20px;">&diams;</a>&emsp;''' + title2 + '''</p>
</div>
<div style="
    position: fixed; 
    bottom: 50px;
    left: 50px;
    width: 150px;
    height: 80px; 
    z-index:9998;
    font-size:14px;
    background-color: #ffffff;

    opacity: 0.7;
    ">
</div>
{% endmacro %}
'''
  legend = branca.element.MacroElement()
  legend._template = branca.element.Template(legend_html)

  for i in subset1:
    s = df[df['Study ID'] == i]
    numRows = len(s)
    if numRows == 0:
      continue
    graphpoints = []
    for f in range(0,numRows):
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]

      tup = [lat,long]

      graphpoints.append(tup)
    folium.PolyLine(locations = graphpoints, color = '#4363d8', tooltip = s.iloc[f,0]).add_to(map2)
  for i in subset2:
    s = df[df['Study ID'] == i]
    numRows = len(s)
    if numRows == 0:
      continue
    graphpoints = []
    for f in range(0,numRows):
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]

      tup = [lat,long]

      graphpoints.append(tup)
    folium.PolyLine(locations = graphpoints, color = '#f032e6', tooltip = s.iloc[f,0]).add_to(map2)
  folium.LayerControl().add_to(map2)
  map2.get_root().add_child(legend)

  return map2

In [ ]:

compMap = compareGroups(df,l,m, 'Low Income', 'Middle Income')

Comparing participants below 80% of county (blue) with particiapants between 80% and  200% of county median median (magenta)

In [ ]:
compMap

Comparing participants who answered yes (magenta) vs no (blue) on " Did financial difficulties ever cause you or your family to move to a difference place (before the age of 16)?" from the childhood SES questionaire.



In [ ]:
noFinDifMove = pd.read_csv(pathe + 'MBES/Study_IDs_Moved_As_A_Result_Of_Hardship.csv')
FinDifMove = pd.read_csv(pathe + 'MBES/Study_IDs_NOT_Moved_As_A_Result_Of_Hardship.csv')
nFDM = noFinDifMove['study_id']
FDM = FinDifMove['study_id']
cSEScompMap = compareGroups(df,nFDM,FDM,'Did NOT report moving due to financial hardship before the age of 16', 'Did report moving due to financial hardship before the age of 16')
print("The geocodable ID ratio of those who answered no is " + str(geocodedratio(df, nFDM)))
print("The geocodable ID  of those who answered yes is " + str(geocodedratio(df, FDM)))
cSEScompMap

# Computing geodesic and binary path length, examples and brief analysis

Compute network path length based on binary edges and geodesic distance moved in miles

In [ ]:


def geographicdistance(df):
  disDF = pd.DataFrame(columns = ['Study ID', 'Distance Traveled', 'Number of Moves'])
  unique = df['Study ID'].unique()
  for i in unique:
    print(i)
    s = df[df['Study ID'] == i]
    numRows = len(s)
    if numRows == 0:
      continue
    graphpoints = []
    for f in range(0,numRows):
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]
      tup = [lat,long]
      graphpoints.append(tup)
    totaldis = 0
    j = 0
    while j < (len(graphpoints) - 1):
        # totaldis = totaldis + dist(graphpoints[j][0],graphpoints[j][1], graphpoints[j+1][0],graphpoints[j+1][1])
        totaldis = totaldis + distance.distance(graphpoints[j], graphpoints[j+1]).miles
        j = j + 1
    df2 = {'Study ID': int(i) , 'Distance Traveled': totaldis, 'Number of Moves': numRows}

    disDF = disDF.append(df2, ignore_index = True)
  return disDF

dis = geographicdistance(df)


Looking at geocoded set statistics

In [ ]:
dis.describe()

In [ ]:
fig = px.box(dis, y= 'Distance Traveled')
fig.show()

In [ ]:
# Allows easy creation of subset using csv of redcap report
def subsetdis(disdf,filepath):
  df = pd.read_csv(filepath)
  IDlist = df['study_id']
  new = disdf[disdf["Study ID"].isin(IDlist)]
  return new

MAASOF = subsetdis(dis,pathe + 'MBES/Study_IDs_Moved_As_A_Result_Of_Hardship.csv')
nMAASOF = subsetdis(dis,pathe + 'MBES/Study_IDs_NOT_Moved_As_A_Result_Of_Hardship.csv')


In [ ]:
# Function that adds a column to distance dataframe tagging each address by group.
# nameofCol is a string representing the name of the column containing the tags
# file1 is the redcap report file (csv) corresponding to group 1
# file2 is the redcap report file (csv) corresponding to group 2
# Value1 is the value of nameofCol for each item in group 1
# Value2 is the value of nameofCol for each item in group 2

def compareGroups(disdf, nameofCol, file1, value1, file2, value2):
  group1 = subsetdis(disdf,file1)
  group2 = subsetdis(disdf,file2)
  disdf[nameofCol] = None
  numRows = len(disdf.index)
  cols = len(disdf.columns)
  for f in range(0,numRows):
    disdf.iloc[f,0] = int(disdf.iloc[f,0])
    if disdf.iloc[f,0] in group1['Study ID'].values:
      disdf.iloc[f,cols -1] = value1
    elif dis.iloc[f,0] in group2['Study ID'].values:
      disdf.iloc[f,cols -1] = value2
  return disdf

In [ ]:

dis = compareGroups(dis, 'Moved as a result of hardship', pathe + 'MBES/Study_IDs_NOT_Moved_As_A_Result_Of_Hardship.csv', 'No',pathe + 'MBES/Study_IDs_Moved_As_A_Result_Of_Hardship.csv' , 'Yes')



Looking at those who answered no to" Did financial difficulties ever cause you or your family to move to a difference place (before the age of 16)?" from the childhood SES questionaire.

In [ ]:
dis['Income'] = None
numRows = len(dis.index)
for f in range(0,numRows):
  dis.iloc[f,0] = int(dis.iloc[f,0])
  if dis.iloc[f,0] in m:
    dis.iloc[f,4] = 'Middle'
  elif dis.iloc[f,0] in l:
    dis.iloc[f,4] = 'Lower'
  else:
    dis.iloc[f,4] = None

dis.describe()

In [ ]:
fig = px.box(dis ,x = 'Moved as a result of hardship', y= 'Distance Traveled')
fig.show()

Possible difference here in median- maybe those who moved due to financil hardship tended to move farther. It would be interesting to see if this relates to social capital, social mobility, and mid life SES

Looking at those who low income, below the 80% county median, and those who are middle income, between 80% and 200% median income



In [ ]:
fig = px.box(dis ,x = 'Income', y= 'Distance Traveled')
fig.show()

In [ ]:
dis.to_csv(pathe+'MBES/ELEMecongeochanges', index = False)

# Comparing Economic Mobility with Geographic Mobility 


In [ ]:
dftract.iloc[0,27]

First, we will need to calculate economic changes for each participant. We will put the result into the same dataframe we put the geographic mobility result in 

In [ ]:
def econchange(dftract, dis):
  dis['Economic Mobility'] = None
  unique = dftract['Study ID'].unique()
  for i in unique:
    s = dftract[dftract['Study ID'] == i]
    numRows = len(s)
    if numRows == 0:
      continue
    econChange = s.iloc[numRows - 1,(27)] - s.iloc[0,(27)]
    dis.iloc[dis['Study ID'] == i,5] = econChange

  return dis

In [ ]:
dis = econchange(dftract, dis)
dis.to_csv(pathe+'MBES/ELEMecongeochanges', index = False)

In [ ]:
dis.head()

In [ ]:
sns.scatterplot(data=dis, x="Economic Mobility", y="Distance Traveled")

# Maps integrated with census data

Combining maps with SES data

In [ ]:
key = apikey.load("census", filename=pathe + 'MBES/secretkey')

In [ ]:
c = Census(key)

In [ ]:
dftract.head()

In [ ]:
dftract.head(500)

In [ ]:
def incomechange(df):
    df['Income Change'] = None
    unique = df['Study ID'].unique()
    numRows = len(df.index)
    for f in range(0,numRows):
        if df.iloc[f,1] == 1: 
            df.iloc[f,28] = 0 
        else: 
            df.iloc[f,28] = df.iloc[f,27] - df.iloc[f-1,27]
    return df
dftract = incomechange(dftract)

In [ ]:
dftract[dftract['Income Change'] == dftract['Income Change'].max()]

In [ ]:
import math
colstr = ['#D8FCD8', '#A0F9A0'  , '#62F163' , '#2AED2D' , '#03DF04']

colstr2 = ['#F8D1CD', '#F0A8AB', '#E97E88', '#E15566','#DA2C43']
colrev = copy.deepcopy(colstr2)
colrev.reverse()
totalcol = colrev + colstr
rgbcol = []
for i in totalcol:    
    rgbcol.append(matplotlib.colors.to_rgb(i))
def incomemap(df):
  map2 = folium.Map(
    location=[37.0902, -95.7129],
     height=1000,
    zoom_start=5,
)
  colormap = branca.colormap.StepColormap(colors = rgbcol,vmin = -179520.0, vmax = 196131.0  )
  #colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
  colormap.caption = 'Change in Median Income As a Result of Moving'
  colormap.add_to(map2)
  unique = df['Study ID'].unique()
  for i in unique:
    s = df[df['Study ID'] == i]
    numRows = len(s)
    for f in range(1,numRows):
      if s.iloc[f,(28)] < 0: 
        cols = colstr2
      else: 
        cols = colstr
      long0 = s.iloc[f-1,(21)]
      lat0  = s.iloc[f-1,(20)]
      long = s.iloc[f,(21)]
      lat  = s.iloc[f,(20)]
      
      ratio =  abs(s.iloc[f,(28)]) / df['Income Change'].max()
      if (long - long0) == 0:
        continue 
      tup = [lat,long]
      mid = ((lat0 + lat)/2, (long0 + long)/2)
      angle = math.degrees(math.atan2(lat - lat0, long - long0))
      # rotation = 90*atan((lat - lat0)/(long - long0))
      folium.PolyLine(locations = [[lat0,long0],[lat,long]], tooltip = s.iloc[f,0], color = cols[int(ratio*4)]).add_to(map2)
      #folium.Marker(location=mid, icon=folium.Icon(color='lightgray', icon='glyphicon glyphicon-chevron-left')).add_to(m)
      # cent = [lat0,long0]
      # folium.RegularPolygonMarker(location= cent, fill_color='blue', number_of_sides=3, radius=10, rotation= rotation).add_to(map2)
  return map2

In [ ]:
incomemap = incomemap(dftract)

In [ ]:
incomemap 